### 🧠 Tech Consulting Proposal Generator with Scoring Feedback Loop

This notebook demonstrates how to use OpenAI's GPT model to:
- Generate a proposal based on client input
- Store and manage conversation context
- Score the output based on a simple rubric
- Feed the score back into the generation loop to improve quality

**You will need your OpenAI API key to run this.**

In [1]:
# 🔧 Setup
import openai
import os

KEY = os.getenv("OPENAI_VPD_API_KEY")
openai.api_key = KEY

#initialize client
client = openai.OpenAI(api_key=KEY)

# Initialize conversation history
conversation = [
    {"role": "system", "content": "You are a tech consultant generating proposals tailored to clients' needs."}
]

In [ ]:
# ➕ Helper function to add a message
def add_message(role, content):
    conversation.append({"role": role, "content": content})

# 🤖 Generate response
def generate_response():
    response = client.response.create(
        model="gpt-4",
        messages=conversation
    )
    reply = response.output_text
    add_message("assistant", reply)
    return reply

In [ ]:
# 📝 Example: Add your prompt about the client
user_input = "Client is a logistics firm seeking a cloud-native inventory management solution."
add_message("user", user_input)
proposal = generate_response()
print(proposal)

In [ ]:
# 📊 Score the proposal using a simple rubric
def score_proposal(proposal):
    rubric = {
        "clarity": lambda p: int(len(p.split('.')) > 10),
        "technical depth": lambda p: int('architecture' in p.lower() or 'API' in p),
        "client relevance": lambda p: int('you' in p.lower() or 'your company' in p)
    }
    category_scores = {k: fn(proposal) for k, fn in rubric.items()}
    total_score = sum(category_scores.values()) / len(rubric) * 10
    return total_score, category_scores

score, breakdown = score_proposal(proposal)
print(f"Total Score: {score:.2f}/10")
print("Breakdown:", breakdown)

In [ ]:
# 🔁 Improve the proposal based on the score
feedback = f"The proposal received the following scores: Clarity: {breakdown['clarity']}/1, Technical: {breakdown['technical depth']}/1, Relevance: {breakdown['client relevance']}/1."
feedback += " Please revise it to improve clarity and relevance."
add_message("user", feedback)
improved_proposal = generate_response()
print(improved_proposal)